# TensorBoard的使用



In [30]:
from torch.utils.tensorboard import SummaryWriter
from collections.abc import Mapping

SummaryWriter用于向指定路径中写入日志。

当启动Tensorboard以后，将会把SummaryWriter写入的内容以图表的形式表示出来，并试图自动拟合。

当使用Writer添加内容的时候，需要一个`tag`属性，该属性可以理解为对应的表，修改`tag`后添加数据，将会在新的表中添加和绘制数据；
反之如果不修改`tag`，那么会把多次写入的内容都添加到同一个图表中。

当存在多个不同的函数的时候，Tensorboard拟合数据可能会出错（会拟合出奇怪的东西来），解决方法如下：

1. 删除logs中的内容，让程序从头开始记录log；
2. 创建SummaryWriter的时候，直接将目标地址设置到`/logs`的子文件夹，对于每个模型测试，都创建一个这样的子文件夹，以区别内容。

注意，当启动Tensorboard的时候，如果logdir设置为总体的logs根目录，将会读取该目录下所有的内容以及子目录的内容。


In [31]:
writer = SummaryWriter('../logs/math')  # 创建一个SummaryWriter实例

# writer.add_image()
# 该函数用于添加一个scalar，其中，tag可以认为是图像的标题、scalar_value可以认为是y轴、global_step是x轴。
# writer.add_scalar(tag='loss', scalar_value=2.0, global_step=5000)
# 
# Tensorboard会试图自动拟合写入的数据。
# 

for i in range(0,100):
    writer.add_scalar(tag='y=2x', scalar_value=3*i, global_step=i)

writer.close()

使用writer添加图片的时候，要求图片的格式必须是`torch.Tensor, numpy.ndarray, or string/blobname`中的一种，
而使用PIL读取的图片，一般是`<class 'PIL.JpegImagePlugin.JpegImageFile'>`，所以不可以直接使用。（这一点会在后面几个代码块说明）

此外，也可以使用numpy型的数据，这样需要使用OpenCV读取图片。



In [32]:
"""
本代码块中，通过PIL读取图片，然后用Numpy将其转换为ndarray格式，传入SummaryWriter。
"""
from PIL import Image
import numpy as np

img_path = '../dataset/hymenoptera_data/train/ants/0013035.jpg'  # 图片路径
img_PIL = Image.open(img_path)  # PIL读取图片
img_arr = np.array(img_PIL)  # 转换为Numpy ndarray格式
print(type(img_arr),img_arr.shape)  # 做一个验证

# 创建一个writer
writer = SummaryWriter('../logs/test')

# 添加图片
# 因为转换来的图片格式与`add_image()`的要求不同，所以需要额外指定`dataformats`参数，详情可以参考函数定义注解。
writer.add_image('ant_image', img_arr, global_step=1, dataformats='HWC')


<class 'numpy.ndarray'> (512, 768, 3)


In [33]:
"""
这个代码块中，使用OpenCV读取图片
"""
import cv2

img_path = '../dataset/hymenoptera_data/train/bees/16838648_415acd9e3f.jpg'  # 图片路径

img_cv = cv2.imread(filename=img_path)  # OpenCV读取图片，读取到的是BGR格式，需要转换回RGB格式
# type(img_cv),img_cv.shape  # 数据格式为HWC，在使用writer的时候也需要额外设置数据格式
print(type(img_cv),img_cv.shape)
img = cv2.cvtColor(img_cv,cv2.COLOR_BGR2RGB)

writer = SummaryWriter('../logs/test_bee')
writer.add_image('bee_image', img, global_step=1, dataformats='HWC')


<class 'numpy.ndarray'> (450, 500, 3)
